# Generating the Map !
On compile une liste de coordonnées que l'on complète d'abord avec les csv, puis les gpx, puis les .tcz 
En parallèle on a une liste qui contient tous les noms de fichiers associés avec le même rang que la liste de coordonnées
On finit par matcher le filename sur le dataframe activity.csv pour associer les metadonnées à la bonne sortie

# Handling CSV files

In [39]:
import simplekml
import pandas as pd
import math
import re
import os

from utils import color_palette, description_activity, activity_nb

In [17]:
# permet de config tous les path d'un coup
person = r"\blondo"

In [18]:
path_2 = r"C:\Users\Guilhem Leprince\repo"
path = r"C:\Users\guilhem.leprince\Documents\strava"
csv_directory = path + r"\data\processed_data" + person + r"\csv"
kml_directory = path + r"\data\processed_data" + person + r"\kml"

In [19]:
# la liste des coordonnées trouvées dans chaque csv. Le rang est le même que le filename
# [
#  [[file_0_lon_1, file_0_lat_1],[file_0_lon_2, file_0_lat_2]....],
#  [[file_1_lon_1, file_1_lat_1],....],
#  ...
# ]
list_of_coor = []

In [20]:
CSV_FILE = re.compile("\\d+.csv")

# la liste qui va contenir l'adresse de tous les fichiers csv à considérer
list_of_csv_files = []


for filename in os.listdir(csv_directory):
    f = os.path.join(csv_directory, filename)
    if os.path.isfile(f) and CSV_FILE.match(filename):
        list_of_csv_files.append(f)

print("A total of " + str(len(list_of_csv_files)) + " csv files found")

A total of 65 csv files found


In [21]:
c=1
l = len(list_of_csv_files)
for csv_file in list_of_csv_files:
    filename = csv_file.split("\\")[-1]
    print("["+ str(c)+"/"+str(l)+"] Processing " + filename)
    df=pd.read_csv(csv_file)
    latitude = df.lat.values.tolist()
    longitude = df.lon.values.tolist()           
    coor=[]
    for i in range(1,len(latitude)):
        if ((type(latitude[i])!= str) and (type(longitude[i])!= str) and math.isnan(longitude[i])==False and math.isnan(latitude[i])==False and abs(longitude[i])>0.01):
            coor.append([longitude[i],latitude[i]])
    list_of_coor.append(coor)
    c+=1

[1/65] Processing 1934211992.csv
[2/65] Processing 2616606372.csv
[3/65] Processing 2624836461.csv
[4/65] Processing 2624837125.csv
[5/65] Processing 2624837818.csv
[6/65] Processing 2636539008.csv
[7/65] Processing 2636713287.csv
[8/65] Processing 2713288879.csv
[9/65] Processing 3312998322.csv
[10/65] Processing 3316816466.csv
[11/65] Processing 3322676058.csv
[12/65] Processing 3332268706.csv
[13/65] Processing 3332270194.csv
[14/65] Processing 3332270464.csv
[15/65] Processing 3335343070.csv
[16/65] Processing 5868262189.csv
[17/65] Processing 6124429425.csv
[18/65] Processing 6893921862.csv
[19/65] Processing 6893922901.csv
[20/65] Processing 6902749287.csv
[21/65] Processing 6949197367.csv
[22/65] Processing 7064076719.csv
[23/65] Processing 7792203154.csv
[24/65] Processing 7799273242.csv
[25/65] Processing 7799273275.csv
[26/65] Processing 7799273324.csv
[27/65] Processing 7816301007.csv
[28/65] Processing 7821011814.csv
[29/65] Processing 7826515668.csv
[30/65] Processing 7903

In [22]:
len(list_of_coor)

65

# Handling GPX files

In [23]:
import gpxpy
import gpxpy.gpx
import re
import os

In [24]:
gpx_directory = path + r"\data" + person + r"\activities\\"

In [25]:
GPX_FILE = re.compile("\\d+.gpx(?!.gz)") # regex pour ne considérer que les gpx décompressés

list_of_gpx_files = []


for filename in os.listdir(gpx_directory):
    f = os.path.join(gpx_directory, filename)
    if os.path.isfile(f) and GPX_FILE.match(filename):
        list_of_gpx_files.append(f)

print("A total of " + str(len(list_of_gpx_files)) + " gpx files found")

A total of 3 gpx files found


In [26]:
c=1
l = len(list_of_gpx_files)
for gpx_file in list_of_gpx_files:
    filename = gpx_file.split("\\")[-1]
    print("["+ str(c)+"/"+str(l)+"] Processing " + filename)
    
    gpx_file = open( gpx_directory + filename, 'r', encoding="utf8")
    gpx = gpxpy.parse(gpx_file)

    coor_gpx =[]
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                coor_gpx.append([point.longitude,point.latitude])
    list_of_coor.append(coor_gpx) 
    gpx_file.close()
    print("--> found exactly -- " + str(len(coor_gpx)) + " -- positions")
    c+=1

[1/3] Processing 2207051826.gpx
--> found exactly -- 7274 -- positions
[2/3] Processing 2437471573.gpx
--> found exactly -- 8696 -- positions
[3/3] Processing 6535411291.gpx
--> found exactly -- 3204 -- positions


In [27]:
print(len(list_of_coor))

68


# Handling TCZ : to be coded

In [28]:
from bs4 import BeautifulSoup

In [29]:
tcx_directory = path + r"\data" + person + r"\activities"

In [30]:
TCX_FILE = re.compile("\\d+.tcx(?!.gz)") # regex pour ne considérer que les tcx décompressés

list_of_tcx_files = []


for filename in os.listdir(tcx_directory):
    f = os.path.join(tcx_directory, filename)
    if os.path.isfile(f) and TCX_FILE.match(filename):
        list_of_tcx_files.append(f)

print("A total of " + str(len(list_of_tcx_files)) + " tcx files found")

A total of 545 tcx files found


In [31]:
c=1
l = len(list_of_tcx_files)
for file in list_of_tcx_files:
    coor_tcx =[]
    print("[" + str(c) + "/"+str(l)+"] Processing "+ file.split('\\')[-1])
    with open(file, 'r') as f:
        data = f.read()
    activity = BeautifulSoup(data, "xml")
    list_of_pos = activity.find_all('Position')
    for pos in list_of_pos:
        coor_tcx.append([float(pos.find("LongitudeDegrees").text), float(pos.find("LatitudeDegrees").text)])
    list_of_coor.append(coor_tcx)
    print("-- found exactly -- " + str(len(coor_tcx)) + " -- positions")
    f.close()
    c+=1

[1/545] Processing 1008440946.tcx
-- found exactly -- 1563 -- positions
[2/545] Processing 1008441012.tcx
-- found exactly -- 3769 -- positions
[3/545] Processing 1023699433.tcx
-- found exactly -- 2360 -- positions
[4/545] Processing 1023699437.tcx
-- found exactly -- 3677 -- positions
[5/545] Processing 1035184590.tcx
-- found exactly -- 1327 -- positions
[6/545] Processing 1035184625.tcx
-- found exactly -- 1306 -- positions
[7/545] Processing 1035184827.tcx
-- found exactly -- 1740 -- positions
[8/545] Processing 1037566551.tcx
-- found exactly -- 3485 -- positions
[9/545] Processing 1088079734.tcx
-- found exactly -- 1398 -- positions
[10/545] Processing 1088079738.tcx
-- found exactly -- 1712 -- positions
[11/545] Processing 1088079759.tcx
-- found exactly -- 1999 -- positions
[12/545] Processing 1132132178.tcx
-- found exactly -- 1123 -- positions
[13/545] Processing 1143464297.tcx
-- found exactly -- 2473 -- positions
[14/545] Processing 1143610355.tcx
-- found exactly -- 4434 

-- found exactly -- 15162 -- positions
[114/545] Processing 2291031874.tcx
-- found exactly -- 7585 -- positions
[115/545] Processing 2291031978.tcx
-- found exactly -- 16265 -- positions
[116/545] Processing 2292077552.tcx
-- found exactly -- 11838 -- positions
[117/545] Processing 2297205419.tcx
-- found exactly -- 11866 -- positions
[118/545] Processing 2307540737.tcx
-- found exactly -- 7397 -- positions
[119/545] Processing 2307540746.tcx
-- found exactly -- 9154 -- positions
[120/545] Processing 2318814867.tcx
-- found exactly -- 1982 -- positions
[121/545] Processing 2323845953.tcx
-- found exactly -- 14425 -- positions
[122/545] Processing 2343403273.tcx
-- found exactly -- 17963 -- positions
[123/545] Processing 2351307491.tcx
-- found exactly -- 25809 -- positions
[124/545] Processing 2354797382.tcx
-- found exactly -- 11611 -- positions
[125/545] Processing 2430805857.tcx
-- found exactly -- 9059 -- positions
[126/545] Processing 2430805911.tcx
-- found exactly -- 16240 -- p

-- found exactly -- 0 -- positions
[225/545] Processing 3481835409.tcx
-- found exactly -- 0 -- positions
[226/545] Processing 3481835411.tcx
-- found exactly -- 0 -- positions
[227/545] Processing 3482473681.tcx
-- found exactly -- 2155 -- positions
[228/545] Processing 3492705215.tcx
-- found exactly -- 0 -- positions
[229/545] Processing 3492705247.tcx
-- found exactly -- 0 -- positions
[230/545] Processing 3492705250.tcx
-- found exactly -- 0 -- positions
[231/545] Processing 3514991263.tcx
-- found exactly -- 0 -- positions
[232/545] Processing 3514991280.tcx
-- found exactly -- 0 -- positions
[233/545] Processing 3514991308.tcx
-- found exactly -- 622 -- positions
[234/545] Processing 3526172508.tcx
-- found exactly -- 0 -- positions
[235/545] Processing 3526172538.tcx
-- found exactly -- 0 -- positions
[236/545] Processing 3526172657.tcx
-- found exactly -- 2062 -- positions
[237/545] Processing 3544893614.tcx
-- found exactly -- 1738 -- positions
[238/545] Processing 3566564945

-- found exactly -- 8280 -- positions
[337/545] Processing 5124494520.tcx
-- found exactly -- 2404 -- positions
[338/545] Processing 5167227679.tcx
-- found exactly -- 14665 -- positions
[339/545] Processing 5167272755.tcx
-- found exactly -- 3728 -- positions
[340/545] Processing 5191622534.tcx
-- found exactly -- 14558 -- positions
[341/545] Processing 5198119935.tcx
-- found exactly -- 3614 -- positions
[342/545] Processing 5222989824.tcx
-- found exactly -- 2272 -- positions
[343/545] Processing 5237410119.tcx
-- found exactly -- 4235 -- positions
[344/545] Processing 5258415486.tcx
-- found exactly -- 6000 -- positions
[345/545] Processing 5296534614.tcx
-- found exactly -- 2910 -- positions
[346/545] Processing 5443781440.tcx
-- found exactly -- 9752 -- positions
[347/545] Processing 5549196557.tcx
-- found exactly -- 1969 -- positions
[348/545] Processing 5549964847.tcx
-- found exactly -- 3694 -- positions
[349/545] Processing 5569018421.tcx
-- found exactly -- 2097 -- position

-- found exactly -- 2767 -- positions
[448/545] Processing 7150424246.tcx
-- found exactly -- 2936 -- positions
[449/545] Processing 7171151815.tcx
-- found exactly -- 1919 -- positions
[450/545] Processing 7177026021.tcx
-- found exactly -- 7733 -- positions
[451/545] Processing 7182831939.tcx
-- found exactly -- 3829 -- positions
[452/545] Processing 7199008952.tcx
-- found exactly -- 2610 -- positions
[453/545] Processing 7221341164.tcx
-- found exactly -- 2972 -- positions
[454/545] Processing 7232104942.tcx
-- found exactly -- 1762 -- positions
[455/545] Processing 7246239779.tcx
-- found exactly -- 1137 -- positions
[456/545] Processing 7246242091.tcx
-- found exactly -- 2368 -- positions
[457/545] Processing 7276305688.tcx
-- found exactly -- 1551 -- positions
[458/545] Processing 7285676046.tcx
-- found exactly -- 11468 -- positions
[459/545] Processing 7292431794.tcx
-- found exactly -- 8333 -- positions
[460/545] Processing 7298731682.tcx
-- found exactly -- 2277 -- positions

In [32]:
len(list_of_coor)

613

# Handling Metadata

In [62]:
metadata_path_2 = r"C:\Users\Guilhem Leprince\repo\data" + person
metadata_path = r"C:\Users\guilhem.leprince\Documents\strava\data" + person

In [63]:
activity_nb = {
    "Randonnée" : 1,    
    "Hike" : 1,
    "Course à pied" : 2,
    "Run" : 2,
    "Ski de randonnée" : 3,
    "Backcountry Ski" : 3,
    "Ski alpin" : 4,
    "Alpine Ski" : 4,
    "Escalade" : 5,
    "Rock Climb": 5,
    "Vélo": 6,
    "Ride" : 6,
    "Marche" : 7,
    "Walk" : 7,
    "Stand Up Paddling" : 8,
    "Rowing" : 9,
    "Virtual Ride" : 10,
    "Ski nordique" : 11,
    "Nordic Ski": 11,
    "Natation" : 12,
    "Swim" : 12,
    "Entraînement" : 13,
    "Workout" :13,
    "Snowshoe": 14,
    "Raquettes" : 14,
    "Yoga" : 15,
    "Aviron" : 16,
    "Fauteuil roulant" : 17}


description_activity= {
    1 : "Hike",
    2 : "Run",
    3 : "Ski de rando",
    4 : "Ski Alpin",
    5 : "Escalade",
    6 : "Vélo",
    7 : "Walk",
    8 : "Stand Up Paddling",
    9 : "Rowing",
    10 : "Virtual Ride",
    11 : "Ski nordique",
    12 : "Natation",
    13 : "Training",
    14 : "Raquette de ses morts",
    15 : "Yoga",
    16 : "Aviron",
    17 : "Fauteuil roulant"
}

color_palette = {
    "" : [255, 255, 255],   # WHITE - null
    1 : [129, 38, 192],     # PURPLE - hike
    2 : [42, 75, 215],      # BLUE - run
    3 : [173, 35, 35],      # RED - ski de rando
    4 : [129, 197, 122],    # LIGHT GREEN - ski alpin
    5 : [255, 255, 255],    # WHITE - escalade
    6 : [255, 238, 51],     # YELLOW - vélo
    7 : [160, 160, 160],    # LIGHT GREY - walk
    8 : [255, 146, 51],     # ORANGE - Stand Up Paddling
    9 : [157, 175, 255],    # LIGHT BLUE - rowing
    10 : [255, 146, 51],    # ORANGE - virtual ride
    11 : [41, 208, 208],    # CYAN - ski nordique
    12 : [87, 87, 87],      # DARK GREY - natation
    13 : [129, 74, 25],     # BROWN - training
    14 : [129, 74, 25],     # BROWN - raquette
    15 : [129, 74, 25],     # BROWN - yoga
    16 : [129, 74, 25],     # BROWN - aviron
    17 : [129, 74, 25]      # BROWN - fauteuil roulant
}

In [65]:
# toutes les metadonnées d'une personne sont stockées dans ce dataframe
df=pd.read_csv(metadata_path + "\\activities.csv")

# renaming columns in french or english
df=df.rename(columns={"ID de l'activité":"ID", "Date de l'activité":"date", "Nom de l'activité":"name",
                      "Type d'activité":"type", "Temps écoulé":"duration", "Distance":"dist", "Nom du fichier":"filename",
                      "Poids de l'athlète":"weight", "Temps écoulé.1":"duration_1", "Durée de déplacement":"duration_mvt",
                      "Distance.1":"dist_1", "Vitesse max.":"max_spd","Dénivelé positif":"d_plus", "Altitude min.":"alti_min",
                      "Altitude max.":"alti_max", "Pente max.":"max_slope", "Pente moyenne":"avg_slope", "Calories":"calories"})
df=df.rename(columns={"Activity ID":"ID", "Activity Date":"date", "Activity Name":"name",
                      "Activity Type":"type", "Elapsed Time":"duration", "Distance":"dist", "Filename":"filename",
                      "Athlete Weight":"weight", "Elapsed Time.1":"duration_1", "Moving Time":"duration_mvt",
                      "Distance.1":"dist_1", "Max Speed":"max_spd","Elevation Gain":"d_plus", "Elevation Low":"alti_min",
                      "Elevation High":"alti_max", "Max Grade":"max_slope", "Average Grade":"avg_slope", "Calories":"calories"})


# filtering columns
df=df[["ID", "date", "name", "type", "duration", "dist",
       "filename", "weight", "duration_1", "duration_mvt", "dist_1", "max_spd",
       "d_plus", "alti_min", "alti_max", "max_slope", "avg_slope", "calories"]]

# Replacing activity name by number
df=df.replace({"type":activity_nb})
df[df.type==17].head()

,ID,date,name,type,duration,dist,filename,weight,duration_1,duration_mvt,dist_1,max_spd,d_plus,alti_min,alti_max,max_slope,avg_slope,calories
90,1569434281,13 mai 2018 à 08:28:02,Raid TOSS finale-Run&Bike début,17,2882,"11,24",activities/1687772584.tcx.gz,70.0,2882.0,2873.0,11240.599609,12.6,240.236786,63.0,171.199997,48.299999,-0.434141,1061.27356


In [66]:
# nettoyage + on extrait le filename du path
# Attention, filename != ID
df.set_index("ID")
list_of_filename = df.filename.values.tolist()
corrected_filename=[]
for i in list_of_filename:
    if type(i)==str and i.__contains__("/"):
        corrected_filename.append(i.split("/")[1].split(".")[0])
    else:
        corrected_filename.append("")
df.filename= corrected_filename
df.ID=df.ID.astype(str)

# Writting kml file 

In [59]:
# Cette liste fait exactement la même taille que celle des coordonnées
list_of_filename = []
list_of_filename = list_of_csv_files + list_of_gpx_files + list_of_tcx_files


In [60]:
# On applique un tri : on supprime sur les sorties dont le nombre de position est inférieur à 4

list_of_notnull_coor_=[]
list_of_notnull_filename=[]

print("Before filtering, we have " + str(len(list_of_coor) - len(list_of_notnull_coor_))+ " activities to plot")

for i in range(len(list_of_coor)):
    if (len(list_of_coor[i]) >= 4) :
        list_of_notnull_coor_.append(list_of_coor[i])
        list_of_notnull_filename.append(list_of_filename[i])
    else:
        print("Removing index " + str(i) + " with a len of " + str(len(list_of_coor[i])))


print("After filtering, we have " + str(len(list_of_notnull_coor_))+ " activities to plot")

Before filtering, we have 613 activities to plot
Removing index 30 with a len of 0
Removing index 141 with a len of 0
Removing index 154 with a len of 0
Removing index 157 with a len of 0
Removing index 158 with a len of 3
Removing index 273 with a len of 0
Removing index 274 with a len of 0
Removing index 275 with a len of 0
Removing index 276 with a len of 0
Removing index 278 with a len of 0
Removing index 279 with a len of 0
Removing index 280 with a len of 0
Removing index 282 with a len of 0
Removing index 283 with a len of 0
Removing index 284 with a len of 0
Removing index 285 with a len of 0
Removing index 286 with a len of 0
Removing index 288 with a len of 0
Removing index 289 with a len of 0
Removing index 290 with a len of 0
Removing index 291 with a len of 0
Removing index 292 with a len of 0
Removing index 293 with a len of 0
Removing index 295 with a len of 0
Removing index 296 with a len of 0
Removing index 297 with a len of 0
Removing index 298 with a len of 0
Removin

In [67]:
kml = simplekml.Kml()

i=0
l = len(list_of_notnull_filename)
c=1
for coor in list_of_notnull_coor_:
    
    print("["+ str(c)+"/"+str(l)+"] added to map")
    # permet d'afficher le marker au niveau du quart de la sortie (milieu de tracé sur un aller-retour)
    p = int(len(coor)/4)
    dplus=""
    namefile = list_of_notnull_filename[i].split("\\")[-1].split(".")[0]
    
    # on match le type d'activité avec la couleur correspondante
    line_color=[]
    line_color=color_palette[df[df.filename==namefile].type.values[0]]

    # on crée la trace : coordonnées + on match le nom
    ls = kml.newlinestring(coords=list_of_notnull_coor_[i], name = str(df[df.filename==namefile].name.values[0]))  

    # on applique couleur et taille
    ls.style.linestyle.color = simplekml.Color.rgb(line_color[0], line_color[1], line_color[2])
    ls.style.linestyle.width = 3
    
    # gestion pas belle du dplus. Faudrait ajouter un test sur sa nullité
    try:
        dplus=str(round(df[df.filename==namefile].d_plus.values[0]))
    except(ValueError):
        pass
    
    # description de la trace
    ls.description = str(df[df.filename==namefile].name.values[0]) + "\n Activity : "+ description_activity[df[df.filename==namefile].type.values[0]]  + "\n Distance : "+ str(df[df.filename==namefile].dist.values[0]) + "\n d+ : "+ dplus + "m"
    
    # on crée le marker
    pnt = kml.newpoint()
    pnt.coords = [(list_of_notnull_coor_[i][p][0], list_of_notnull_coor_[i][p][1])]
    pnt.description = str(df[df.filename==namefile].name.values[0]) + "\n Activity : "+ description_activity[df[df.filename==namefile].type.values[0]]  + "\n Distance : "+ str(df[df.filename==namefile].dist.values[0]) + "\n d+ : "+ dplus + "m"
    pnt.style.labelstyle.scale = 0.7
    pnt.style.labelstyle.color = simplekml.Color.blue
    c+=1
    i+=1
    
# enregistrement du fichier
kml.save(kml_directory + r""+ person + r"_tracemap_complete.kml")

[1/576] added to map
[2/576] added to map
[3/576] added to map
[4/576] added to map
[5/576] added to map
[6/576] added to map
[7/576] added to map
[8/576] added to map
[9/576] added to map
[10/576] added to map
[11/576] added to map
[12/576] added to map
[13/576] added to map
[14/576] added to map
[15/576] added to map
[16/576] added to map
[17/576] added to map
[18/576] added to map
[19/576] added to map
[20/576] added to map
[21/576] added to map
[22/576] added to map
[23/576] added to map
[24/576] added to map
[25/576] added to map
[26/576] added to map
[27/576] added to map
[28/576] added to map
[29/576] added to map
[30/576] added to map
[31/576] added to map
[32/576] added to map
[33/576] added to map
[34/576] added to map
[35/576] added to map
[36/576] added to map
[37/576] added to map
[38/576] added to map
[39/576] added to map
[40/576] added to map
[41/576] added to map
[42/576] added to map
[43/576] added to map
[44/576] added to map
[45/576] added to map
[46/576] added to m

[394/576] added to map
[395/576] added to map
[396/576] added to map
[397/576] added to map
[398/576] added to map
[399/576] added to map
[400/576] added to map
[401/576] added to map
[402/576] added to map
[403/576] added to map
[404/576] added to map
[405/576] added to map
[406/576] added to map
[407/576] added to map
[408/576] added to map
[409/576] added to map
[410/576] added to map
[411/576] added to map
[412/576] added to map
[413/576] added to map
[414/576] added to map
[415/576] added to map
[416/576] added to map
[417/576] added to map
[418/576] added to map
[419/576] added to map
[420/576] added to map
[421/576] added to map
[422/576] added to map
[423/576] added to map
[424/576] added to map
[425/576] added to map
[426/576] added to map
[427/576] added to map
[428/576] added to map
[429/576] added to map
[430/576] added to map
[431/576] added to map
[432/576] added to map
[433/576] added to map
[434/576] added to map
[435/576] added to map
[436/576] added to map
[437/576] a